## Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Environment: Python 3.10.4 (ipykernel)
#### Last Update: 2022-05-30
#### Description: Web crawling code for EURONEXT.com, especially for EURONEXT Oslo. Can be reused for other country by simple editing.

### Utilized BS4 and Selenium Web for Crawling - Visualized by Pandas

In [1]:
import bs4
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import csv 
import pandas as pd 
from platform import python_version

In [2]:
path = r'C:\Users\DT\Desktop\자료\ERUONEXT\\chromedriver'

In [3]:
print(python_version())

3.10.4


In [4]:
print(pd.__version__)

1.4.2


### Basic setting for csv file that will store 재무정보 data - Order of rows are based on 재무정보 데이터소스

In [5]:
cover = open('NorwayData.csv','w', newline='', encoding='CP949')
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드', '현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

### Crawled Website
EURONEXT MAIN: https://live.euronext.com/en /
EURONEXT Oslo: https://live.euronext.com/markets/oslo/equities/list

## This is the main code that crawls data rom EURONEXT. 
#### - Time sleep was necessary because of comparably long loading time.
#### - Uncollectable or unavailable data was displayed as "(NA) DataName."
#### - (계산예정) Data Will be calculated and inserted manually later.
#### - Sample Visualization is located in the bottom.

In [6]:
failList = []
#EURONEXT 
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/en/markets/oslo/equities/list')
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[3]/div/div/div[3]/button[1]').click()
time.sleep(1)
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/button').click()
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[2]/div/div[1]/div/div/div[2]/form/div/div/div[3]/div/div[1]/div[1]/div[2]/button/div').click()
time.sleep(1)
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[2]/div/div[1]/div/div/div[2]/form/div/div/div[3]/div/div[1]/div[2]/div/div[1]/div/div/label').click()
time.sleep(1)
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[2]/div/div[1]/div/div/div[2]/form/div/div/div[4]/input').click()
time.sleep(2.2)
base_url = "https://live.euronext.com"


#Get each company's link, market symbol, and market code on table - P1
time.sleep(2)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
all_market_maintable = maintable.find_all(attrs={'td',"stocks-market"})
all_symbol_maintable = maintable.find_all(attrs={'td',"stocks-symbol"})

#We have to process the collected data again because of their special structure
newList3 =[]
for a in all_market_maintable :
    newList3.append(a.text)
newList4 =[]
for b in all_symbol_maintable :
    newList4.append(b.text)

#Get each company's link, market symbol, and market code on table - P2
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/span/a[2]').click()
time.sleep(2)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable2 = soup.find('table', id='stocks-data-table')
time.sleep(2)
all_atag_maintable2 = maintable2.find_all('a')
all_market_maintable2 = maintable2.find_all(attrs={'td',"nowrap pointer"})
all_symbol_maintable2 = maintable2.find_all(attrs={'td',"stocks-symbol"})

#We have to process the collected data again because of their special structure
newList1 =[]
for c in all_market_maintable2 :
    newList1.append(c.text)

newList2 =[]
for d in all_symbol_maintable2 :
    newList2.append(d.text)
    
#Get each company's link, market symbol, and market code on table - P2
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/span/a[3]').click()
time.sleep(2)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable3 = soup.find('table', id='stocks-data-table')
time.sleep(2)
all_atag_maintable3 = maintable3.find_all('a')
all_market_maintable3 = maintable3.find_all(attrs={'td',"nowrap pointer"})
all_symbol_maintable3 = maintable3.find_all(attrs={'td',"stocks-symbol"})

#We have to process the collected data again because of their special structure
newList5 =[]
for c in all_market_maintable3 :
    newList5.append(c.text)

newList6 =[]
for d in all_symbol_maintable3 :
    newList6.append(d.text)
    
#Merge two lists (from page 1 and 2)
all_atag_maintable = all_atag_maintable + all_atag_maintable2
all_symbol_maintable = newList3 + newList1 + newList6
all_market_maintable = newList4 + newList2 + newList5

#Final Processing to remove unnecessary data
all_symbol_maintable.pop(0)
all_market_maintable.pop(0)
company_links = []
failList = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    

# To check if all companies are selected
#print(len(company_links))

# Main Data Collection code
for sub in company_links:
    driver.get(base_url + sub)
    info = []
    time.sleep(2)
    
    #국가코드
    info.append("NO")
    info.append("Norge")
    info.append("Norway")
    info.append("UTC+01:00")
    info.append("Europe")
    info.append("36600000000 USD")
    info.append("5425270")
    info.append("북유럽")
    
    #기업
    name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
    info.append("NLD XOSL" + name)
    info.append(name)
    info.append(name)
    
    #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
    try :
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
        time.sleep(2)
        address = driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[1]/div[2]').text
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norge). Address: " + address)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norway). Address: " + address)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norge). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K  Norwegian Krones. You can find more information on company's website. Company Address: " + address)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norway). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K  Norwegian Krones. You can find more information on company's website. Company Address: " + address)
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norge).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norway).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norge). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norway). You can find more information on company's website.")
        info.append("")
        info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text) 
        except:
            try: 
                info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[3]/a').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[3]/div/a').text)
        except:
            info.append(str(base_url + sub))
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
        except:
            ("Norge")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
        except:
            info.append("Norway")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
        except:
            info.append("Norge, Europe")
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
        except:
            info.append("Norway, Europe")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("Active") 
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
        except:
            info.append("")
            info.append("")
        info.append("CONTACT")
        info.append("CONTACT")
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[1]/div[1]/h1/strong').text)
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
        except:
            try: info.append(driver.find_elemen(By.XPATH, value ='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/address/div[2]/div[1]/a').text)
            except:
                info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
        except:
            info.append("")  
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
        except:
            try: 
                info.append(driver.find_elemen(By.XPATH, value ='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[1]/a').text)
            except:
                info.append("")
    except: 
        ## 소개내용
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norge).")
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norway).")
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norge). You can find more information on company's website.")
        info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT Oslo(Norway). You can find more information on company's website.")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(str(base_url + sub))
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("Norge")
        info.append("Norway")
        info.append("Norge, Europe")
        info.append("Norway, Europe")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("Active") 
        info.append("")
        info.append("")
        info.append("CONTACT")
        info.append("CONTACT")
        info.append("")
        info.append("")
        info.append("")
        info.append(str(base_url + sub))
        info.append("")
        
    #재무정보
    try:
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
        time.sleep(2)
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/thead/tr/th[2]').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[13]/td').text)
            except:
                info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "천 노르웨이 크로네")
        except:
            info.append("")  
        info.append("")
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[2]').text + "천 노르웨이 크로네")
        except:
            info.append("")  
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[2]').text + "천 노르웨이 크로네")
        except:
            info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[7]/td[2]').text + "천 노르웨이 크로네")      
        except:
            info.append("") 
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
    except : 
        info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")  
        info.append("")
        info.append("")
        info.append("")  
        info.append("")
        info.append("")  
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("") 
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

    #재무정보중 계산예정            
    info.append("")
    info.append("")
    info.append("")
    info.append("")
    info.append("")
    
    #산업군
    try:
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
        time.sleep(2)
        try : 
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
        except :
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
            except:
                info.append("")
                info.append("")
        try :
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
            info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
        except :
            try: 
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
            except:
                info.append("")
                info.append("")
    except:
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        
    #언어
    info.append("NNO")
    info.append("Bokmal")
    info.append("Norwegian")
    
    #주식시장정보
    info.append(str(all_symbol_maintable.pop(0)))
    info.append("EURONEXT Oslo")
    info.append("EURONEXT Oslo")
    info.append(str(all_market_maintable.pop(0)))
    info.append("")
    driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[1]').click()
    time.sleep(2)
    try:
        info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-price').text + " NOK")
    except:
        info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[2]').text  + " NOK")
    except:
        info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[3]').text  + " NOK")
    except:
        info.append("")
    info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[5]').text  + " NOK")
    except:
        info.append("")
    info.append("")
    info.append("")
    info.append("")
    
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[3]/td[2]').text)
    except:
        info.append("")
    try:
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[12]/td[2]').text)
    except:
        info.append("")

    #지점
    info.append("")
    info.append("")
    info.append("Norway")
    info.append("Norway, Europe")
    info.append("")
    info.append("")
    
    #이벤트
    try:
        element = driver.find_element(By.CSS_SELECTOR, value='#content-left > section > div.container-full > div.card.mb-4.text-left > div.card-body > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a')
        href = element.get_attribute('href')
        info.append(href)
    except:
        info.append("")
        
    #화폐
    info.append("NOK")
    info.append("노르웨이 크로네")
    
    #관리
    info.append("Chris")
    info.append("EURONEXT")
    info.append("2022-05-30")
    
    #Keep data in csv
    print(info)
    try: 
        writing.writerow(info);
    except:
        failList.append(info);

C:\Users\DT\AppData\Local\Temp\ipykernel_3992\2674363931.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


['NO', 'Norge', 'Norway', 'UTC+01:00', 'Europe', '36600000000 USD', '5425270', '북유럽', 'NLD XOSL2020 BULKERS', '2020 BULKERS', '2020 BULKERS', '2020 BULKERS(English: 2020 BULKERS) is a public company that is listed on EURONEXT Oslo(Norge). Address: S.E. Pearman Bldg., 2nd Floor, 9 Par-la-Ville Road', '2020 BULKERS(English: 2020 BULKERS) is a public company that is listed on EURONEXT Oslo(Norway). Address: S.E. Pearman Bldg., 2nd Floor, 9 Par-la-Ville Road', '2020 BULKERS(English: 2020 BULKERS) is a public company that is listed on EURONEXT Oslo(Norge).', '2020 BULKERS(English: 2020 BULKERS) is a public company that is listed on EURONEXT Oslo(Norway).', "2020 BULKERS(English: 2020 BULKERS) is a public company that is listed on EURONEXT Oslo(Norge). You can find more information on company's website.", "2020 BULKERS(English: 2020 BULKERS) is a public company that is listed on EURONEXT Oslo(Norway). You can find more information on company's website.", '', '', '', '', '+1 (441) 737-0152', 

['NO', 'Norge', 'Norway', 'UTC+01:00', 'Europe', '36600000000 USD', '5425270', '북유럽', 'NLD XOSLAKER', 'AKER', 'AKER', 'AKER(English: AKER) is a public company that is listed on EURONEXT Oslo(Norge). Address: P.O Box 243', 'AKER(English: AKER) is a public company that is listed on EURONEXT Oslo(Norway). Address: P.O Box 243', 'AKER(English: AKER) is a public company that is listed on EURONEXT Oslo(Norge).', 'AKER(English: AKER) is a public company that is listed on EURONEXT Oslo(Norway).', "AKER(English: AKER) is a public company that is listed on EURONEXT Oslo(Norge). You can find more information on company's website.", "AKER(English: AKER) is a public company that is listed on EURONEXT Oslo(Norway). You can find more information on company's website.", '', '', '', '', '+47 24 13 00 00', '', 'www.akerasa.com', '', '', '', '', '', 'P.O Box 243', 'P.O Box 243', '1326 Lysaker', '1326 Lysaker', '', '', '', '', 'Active', '', '', 'CONTACT', 'CONTACT', 'AKER', '', '+47 24 13 00 00', '', '', 

['NO', 'Norge', 'Norway', 'UTC+01:00', 'Europe', '36600000000 USD', '5425270', '북유럽', 'NLD XOSLAKER SOLUTIONS', 'AKER SOLUTIONS', 'AKER SOLUTIONS', 'AKER SOLUTIONS(English: AKER SOLUTIONS) is a public company that is listed on EURONEXT Oslo(Norge). Address: PO Box 94', 'AKER SOLUTIONS(English: AKER SOLUTIONS) is a public company that is listed on EURONEXT Oslo(Norway). Address: PO Box 94', 'AKER SOLUTIONS(English: AKER SOLUTIONS) is a public company that is listed on EURONEXT Oslo(Norge).', 'AKER SOLUTIONS(English: AKER SOLUTIONS) is a public company that is listed on EURONEXT Oslo(Norway).', "AKER SOLUTIONS(English: AKER SOLUTIONS) is a public company that is listed on EURONEXT Oslo(Norge). You can find more information on company's website.", "AKER SOLUTIONS(English: AKER SOLUTIONS) is a public company that is listed on EURONEXT Oslo(Norway). You can find more information on company's website.", '', '', '', '', '+47 67 51 30 00', '', 'www.akersolutions.com', '', '', '', '', '', 'PO B

['NO', 'Norge', 'Norway', 'UTC+01:00', 'Europe', '36600000000 USD', '5425270', '북유럽', 'NLD XOSLARCTICZYMES TECHNOLOGIES', 'ARCTICZYMES TECHNOLOGIES', 'ARCTICZYMES TECHNOLOGIES', 'ARCTICZYMES TECHNOLOGIES(English: ARCTICZYMES TECHNOLOGIES) is a public company that is listed on EURONEXT Oslo(Norge).', 'ARCTICZYMES TECHNOLOGIES(English: ARCTICZYMES TECHNOLOGIES) is a public company that is listed on EURONEXT Oslo(Norway).', "ARCTICZYMES TECHNOLOGIES(English: ARCTICZYMES TECHNOLOGIES) is a public company that is listed on EURONEXT Oslo(Norge). You can find more information on company's website.", "ARCTICZYMES TECHNOLOGIES(English: ARCTICZYMES TECHNOLOGIES) is a public company that is listed on EURONEXT Oslo(Norway). You can find more information on company's website.", '', '', '', '', '', '', 'https://live.euronext.com/en/product/equities/NO0010014632-XOSL/arcticzymes-techno/azt/quotes', '', '', '', '', '', 'Norge', 'Norway', 'Norge, Europe', 'Norway, Europe', '', '', '', '', 'Active', '',

...

['NO', 'Norge', 'Norway', 'UTC+01:00', 'Europe', '36600000000 USD', '5425270', '북유럽', 'NLD XOSLHEXAGON COMPOSITES', 'HEXAGON COMPOSITES', 'HEXAGON COMPOSITES', 'HEXAGON COMPOSITES(English: HEXAGON COMPOSITES) is a public company that is listed on EURONEXT Oslo(Norge). Address: Postboks 836 Sentrum', 'HEXAGON COMPOSITES(English: HEXAGON COMPOSITES) is a public company that is listed on EURONEXT Oslo(Norway). Address: Postboks 836 Sentrum', 'HEXAGON COMPOSITES(English: HEXAGON COMPOSITES) is a public company that is listed on EURONEXT Oslo(Norge).', 'HEXAGON COMPOSITES(English: HEXAGON COMPOSITES) is a public company that is listed on EURONEXT Oslo(Norway).', "HEXAGON COMPOSITES(English: HEXAGON COMPOSITES) is a public company that is listed on EURONEXT Oslo(Norge). You can find more information on company's website.", "HEXAGON COMPOSITES(English: HEXAGON COMPOSITES) is a public company that is listed on EURONEXT Oslo(Norway). You can find more information on company's website.", '', '',

['NO', 'Norge', 'Norway', 'UTC+01:00', 'Europe', '36600000000 USD', '5425270', '북유럽', 'NLD XOSLINTEROIL EXPLORATION AND PRODUCTION', 'INTEROIL EXPLORATION AND PRODUCTION', 'INTEROIL EXPLORATION AND PRODUCTION', 'INTEROIL EXPLORATION AND PRODUCTION(English: INTEROIL EXPLORATION AND PRODUCTION) is a public company that is listed on EURONEXT Oslo(Norge). Address: Ruseløkkveien 14', 'INTEROIL EXPLORATION AND PRODUCTION(English: INTEROIL EXPLORATION AND PRODUCTION) is a public company that is listed on EURONEXT Oslo(Norway). Address: Ruseløkkveien 14', 'INTEROIL EXPLORATION AND PRODUCTION(English: INTEROIL EXPLORATION AND PRODUCTION) is a public company that is listed on EURONEXT Oslo(Norge).', 'INTEROIL EXPLORATION AND PRODUCTION(English: INTEROIL EXPLORATION AND PRODUCTION) is a public company that is listed on EURONEXT Oslo(Norway).', "INTEROIL EXPLORATION AND PRODUCTION(English: INTEROIL EXPLORATION AND PRODUCTION) is a public company that is listed on EURONEXT Oslo(Norge). You can find

['NO', 'Norge', 'Norway', 'UTC+01:00', 'Europe', '36600000000 USD', '5425270', '북유럽', 'NLD XOSLKID', 'KID', 'KID', 'KID(English: KID) is a public company that is listed on EURONEXT Oslo(Norge). Address: Postboks 505', 'KID(English: KID) is a public company that is listed on EURONEXT Oslo(Norway). Address: Postboks 505', 'KID(English: KID) is a public company that is listed on EURONEXT Oslo(Norge).', 'KID(English: KID) is a public company that is listed on EURONEXT Oslo(Norway).', "KID(English: KID) is a public company that is listed on EURONEXT Oslo(Norge). You can find more information on company's website.", "KID(English: KID) is a public company that is listed on EURONEXT Oslo(Norway). You can find more information on company's website.", '', '', '', '', '94026000', '', 'www.kid.no', '', '', '', '', '', 'Postboks 505', 'Postboks 505', '3412 Lierstranda', '3412 Lierstranda', '', '', '', '', 'Active', '', '', 'CONTACT', 'CONTACT', 'KID', '', '94026000', '', '', '', '', '', '', '', '',

['NO', 'Norge', 'Norway', 'UTC+01:00', 'Europe', '36600000000 USD', '5425270', '북유럽', 'NLD XOSLKOMPLETT BANK', 'KOMPLETT BANK', 'KOMPLETT BANK', 'KOMPLETT BANK(English: KOMPLETT BANK) is a public company that is listed on EURONEXT Oslo(Norge). Address: Postboks 448', 'KOMPLETT BANK(English: KOMPLETT BANK) is a public company that is listed on EURONEXT Oslo(Norway). Address: Postboks 448', 'KOMPLETT BANK(English: KOMPLETT BANK) is a public company that is listed on EURONEXT Oslo(Norge).', 'KOMPLETT BANK(English: KOMPLETT BANK) is a public company that is listed on EURONEXT Oslo(Norway).', "KOMPLETT BANK(English: KOMPLETT BANK) is a public company that is listed on EURONEXT Oslo(Norge). You can find more information on company's website.", "KOMPLETT BANK(English: KOMPLETT BANK) is a public company that is listed on EURONEXT Oslo(Norway). You can find more information on company's website.", '', '', '', '', '+47 21 00 74 50', '', 'www.komplettbank.no', '', '', '', '', '', 'Postboks 448', 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#header-instrument-name > strong"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C0B8F3+2406643]
	Ordinal0 [0x00B9AF31+1945393]
	Ordinal0 [0x00A8C748+837448]
	Ordinal0 [0x00AB92E0+1020640]
	Ordinal0 [0x00AB957B+1021307]
	Ordinal0 [0x00AE6372+1205106]
	Ordinal0 [0x00AD42C4+1131204]
	Ordinal0 [0x00AE4682+1197698]
	Ordinal0 [0x00AD4096+1130646]
	Ordinal0 [0x00AAE636+976438]
	Ordinal0 [0x00AAF546+980294]
	GetHandleVerifier [0x00E79612+2498066]
	GetHandleVerifier [0x00E6C920+2445600]
	GetHandleVerifier [0x00CA4F2A+579370]
	GetHandleVerifier [0x00CA3D36+574774]
	Ordinal0 [0x00BA1C0B+1973259]
	Ordinal0 [0x00BA6688+1992328]
	Ordinal0 [0x00BA6775+1992565]
	Ordinal0 [0x00BAF8D1+2029777]
	BaseThreadInitThunk [0x75C4FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77027A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77027A4E+238]


In [7]:
with open("NorwayData.csv", "r") as table:
    for row in table:
        print(row)

헤브론스타국가코드,현지언어국가명,영문국가명,시간,대륙,GDP,인구,지역,기업식별코드,현지언어기업명,영문기업명,현지언어한줄소개내용,영문한줄소개내용,현지언어기업소개내용,영문기업소개내용,설립일자,법인등록번호,사업자등록번호,기업대표전화번호,대표팩스번호,대표이메일,기업홈페이지URL,페이스북URL,인스타그램URL,유튜브URL,링크드인URL,트위터핸들,현지언어기업주소,영문기업주소,현지언어기업상세주소,영문기업상세주소,기업우편번호,기업종업원,외감법인구분,기업연수,기업상태,현지언어담당자명,영문담당자명,현지언어직위명,영문직위명,담당자부서명,담당자전화번호,담당자팩스번호,담당자이메일,담당자이동전화번호,회계연도,유동자산금액,비유동자산금액,자산총계금액,유동부채금액,비유동부채금액,부채총계금액,자본총계금액,부채자본총계금액,매출액,매출원가금액,판매비관리비금액,영업이익손실금액,금융수익금액,금융비용금액,기타영업외수익금액,기타영업외비용금액,법인세차감전순이익,법인세비용,당기순이익,영업활동현금흐름금액,투자활동현금흐름금액,재무활동현금흐름금액,기초현금자산금액,기말현금자산금액,부채비율,영업이익율,매출액증가율,영업이익증가율,당기순이익 증가율,기업 CAGR,현지언어산업군명,영문산업군명,현지언어주요제품명내용,영문주요제품명내용,국가언어코드,현지언어언어명,영문언어명,주식시장코드,현지언어주식시장명,영문주식시장명,상장코드,상장일자,주가(일),주가(1주),주가(1개월),주가(6개월),주가(1년),주가(3년),주가(5년),주가(10년),거래량,시가총액,지점코드,지점명,주소,주소상세,우편번호,사업자등록번호,이벤트,통화구분코드,화폐단위명,담당자,소스,날짜

NO,Norge,Norway,UTC+01:00,Europe,36600000000 USD,5425270,북유럽,NLD XOSL2020 BULKERS,2020 BULKERS,2020 BULKERS,"2020 BULKERS(English: 2020 BULKERS) is a public company that is listed on EURONEXT Oslo(Norge). Addr

In [8]:
file = open("NorwayData.csv")
csvreader = csv.reader(file)
header = next(csvreader)
print(header)

['헤브론스타국가코드', '현지언어국가명', '영문국가명', '시간', '대륙', 'GDP', '인구', '지역', '기업식별코드', '현지언어기업명', '영문기업명', '현지언어한줄소개내용', '영문한줄소개내용', '현지언어기업소개내용', '영문기업소개내용', '설립일자', '법인등록번호', '사업자등록번호', '기업대표전화번호', '대표팩스번호', '대표이메일', '기업홈페이지URL', '페이스북URL', '인스타그램URL', '유튜브URL', '링크드인URL', '트위터핸들', '현지언어기업주소', '영문기업주소', '현지언어기업상세주소', '영문기업상세주소', '기업우편번호', '기업종업원', '외감법인구분', '기업연수', '기업상태', '현지언어담당자명', '영문담당자명', '현지언어직위명', '영문직위명', '담당자부서명', '담당자전화번호', '담당자팩스번호', '담당자이메일', '담당자이동전화번호', '회계연도', '유동자산금액', '비유동자산금액', '자산총계금액', '유동부채금액', '비유동부채금액', '부채총계금액', '자본총계금액', '부채자본총계금액', '매출액', '매출원가금액', '판매비관리비금액', '영업이익손실금액', '금융수익금액', '금융비용금액', '기타영업외수익금액', '기타영업외비용금액', '법인세차감전순이익', '법인세비용', '당기순이익', '영업활동현금흐름금액', '투자활동현금흐름금액', '재무활동현금흐름금액', '기초현금자산금액', '기말현금자산금액', '부채비율', '영업이익율', '매출액증가율', '영업이익증가율', '당기순이익 증가율', '기업 CAGR', '현지언어산업군명', '영문산업군명', '현지언어주요제품명내용', '영문주요제품명내용', '국가언어코드', '현지언어언어명', '영문언어명', '주식시장코드', '현지언어주식시장명', '영문주식시장명', '상장코드', '상장일자', '주가(일)', '주가(1주)', '주가(1개월)', '주가(6개월)', '주가(1년)', '주가(3년)', '주가(5년)'